In [ ]:
import xarray as xr
import numpy as np
import scipy
import pandas as pd
import statsmodels.api as sm
import scipy
from statsmodels.stats.stattools import durbin_watson

In [ ]:
datapath='/PATH/TO/DATA/'
savepath='/PATH/TO/OUTPUT/'

In [ ]:
#obs
obs5d=xr.open_dataset(datapath+'obs_eke_5day_nol_2016-2020.nc')

In [ ]:
#cmip ensemble members 5day means
c50s5=[]
c15s5=[]
c90s5=[]
for run in [1,2,3,4,5]:
    c50s5.append(xr.open_dataset(datapath+'c6_eke_5d_nol_r'+str(run)+'_reg_1951-1955.nc'))
    c15s5.append(xr.open_dataset(datapath+'c6_eke_5d_nol_r'+str(run)+'_reg_2016-2020.nc'))
    c90s5.append(xr.open_dataset(datapath+'c6_eke_5d_nol_r'+str(run)+'_reg_2091-2095.nc'))

In [ ]:
#so3 5day means
f50=[]
f15=[]
f90=[]
for i in np.arange(5):
    f50.append(datapath+'eke_5d_nol_reg_'+str(1951+i)+'.nc')
    f15.append(datapath+'eke_5d_nol_reg_'+str(2016+i)+'.nc')
    f90.append(datapath+'eke_5d_nol_reg_'+str(2091+i)+'.nc')

seke5_ds50=xr.open_mfdataset(f50)
seke5_ds15=xr.open_mfdataset(f15)
seke5_ds90=xr.open_mfdataset(f90)

In [ ]:
#area mask
accu_mask=np.ones(shape=(1440,720))
accu_mask[:,25*4:45*4]=0
accu_mask[123*4:151*4,int(40*4):]=1

In [ ]:
# area_at_eq
lat_eq=110574/4
lon_eq=111320/4

In [ ]:
cosweights=np.cos(np.radians(obs5d.latitude)).values
cos_lon_d=cosweights*lon_eq
areas=cos_lon_d*lat_eq

In [ ]:
#weight data 

so3_5w50=seke5_ds50.eke.values*areas
so3_5w15=seke5_ds15.eke.values*areas
so3_5w90=seke5_ds90.eke.values*areas

obs5_w=obs5d.ugos*areas.reshape(1,720,1)

c50s5mw=[]
c15s5mw=[]
c90s5mw=[]
for i in np.arange(5):
    c50s5mw.append(c50s5[i].eke.values*areas)
    c15s5mw.append(c15s5[i].eke.values*areas)
    c90s5mw.append(c90s5[i].eke.values*areas)

In [ ]:
#calculate area integrated eke

s5accu50=np.nansum(so3_5w50[:,accu_mask==0],axis=1)
s5accu15=np.nansum(so3_5w15[:,accu_mask==0],axis=1)
s5accu90=np.nansum(so3_5w90[:,accu_mask==0],axis=1)

oaccu5=np.nansum(obs5_w.values[:,accu_mask.T==0],axis=1)
    
c50s5_accu=[]
c15s5_accu=[]
c90s5_accu=[]
for i in np.arange(5):
    c50s5_accu.append(np.nansum(c50s5mw[i][:,accu_mask==0],axis=1))
    c15s5_accu.append(np.nansum(c15s5mw[i][:,accu_mask==0],axis=1))
    c90s5_accu.append(np.nansum(c90s5mw[i][:,accu_mask==0],axis=1))

In [ ]:
#detrend

In [ ]:
s5accu50dt=scipy.signal.detrend(s5accu50)+np.mean(s5accu50)
s5accu15dt=scipy.signal.detrend(s5accu15)+np.mean(s5accu15)
s5accu90dt=scipy.signal.detrend(s5accu90)+np.mean(s5accu90)

oaccu5dt=scipy.signal.detrend(oaccu5)+np.mean(oaccu5)

for i in np.arange(5):
    c50s5_accu=scipy.signal.detrend(c50s5_accu)+np.mean(c50s5_accu)
    c15s5_accu=scipy.signal.detrend(c15s5_accu)+np.mean(c15s5_accu)
    c90s5_accu=scipy.signal.detrend(c90s5_accu)+np.mean(c90s5_accu)

In [ ]:
stats=pd.DataFrame()
stats['Data']=['Ens. 1','Ens. 2','Ens. 3','Ens. 4','Ens. 5','Ens','SO3','Obs.']

stats['Durbin-Watson']=np.nan

stats['Skew']=np.nan
stats['p(Skew)']=np.nan

stats['Kurtosis']=np.nan
stats['p(Kurt)']=np.nan

In [ ]:
# prepare stats for 1951-1955 period
stats50=stats[:7].copy()
for i in np.arange(5):
    #cmip ensemble stats
    ind = np.arange(len(c50s5_accudt[i]))  
    val = c50s5_accudt[i]   
    ind = sm.add_constant(ind)
    reg = sm.OLS(val, ind).fit()
    stats50['Durbin-Watson'][i]=np.round(durbin_watson(reg.resid),3)
    
    stats50['Kurtosis'][i]=np.round(st.kurtosis(c50s5_accudt[i]),3)
    if np.round(st.kurtosistest(c50s5_accudt[i])[1],3) < 0.001:
        stats50['p(Kurt)'][i]='p<0.001'
    else:
        stats50['p(Kurt)'][i]=np.round(st.kurtosistest(c50s5_accudt[i])[1],3)
    stats50['Skew'][i]=np.round(st.skew(c50s5_accudt[i]),3)
    if np.round(st.skewtest(c50s5_accudt[i])[1],3) < 0.001:
        stats50['p(Skew)'][i]='p<0.001'
    else:
        stats50['p(Skew)'][i]=np.round(st.skewtest(c50s5_accudt[i])[1],3)

#ensemble stats
ind = ind = np.tile(np.arange(365),5)
val = cm50ens   
ind = sm.add_constant(ind)
reg = sm.OLS(val, ind).fit()
stats50['Durbin-Watson'][5]=np.round(durbin_watson(reg.resid),3)

stats50['Kurtosis'][5]=np.round(st.kurtosis(cm50ens),3)
if np.round(st.kurtosistest(cm50ens)[1],3) < 0.001:
    stats50['p(Kurt)'][5]='p<0.001'
else:
    stats50['p(Kurt)'][5]=np.round(st.kurtosistest(cm50ens)[1],3)

stats50['Skew'][5]=np.round(st.skew(cm50ens),3)
if np.round(st.skewtest(cm50ens)[1],3) < 0.001:
    stats50['p(Skew)'][5]='p<0.001'
else:
    stats50['p(Skew)'][5]=np.round(st.skewtest(cm50ens)[1],3)
    
#SO3
ind = np.arange(len(s5accu50dt))  
val = s5accu50dt
ind = sm.add_constant(ind)
reg = sm.OLS(val, ind).fit()
stats50['Durbin-Watson'][6]=np.round(durbin_watson(reg.resid),3)

stats50['Kurtosis'][6]=np.round(st.kurtosis(s5accu50dt),3)
if np.round(st.kurtosistest(s5accu50dt)[1],3) < 0.001:
    stats50['p(Kurt)'][6]='p<0.001'
else:
    stats50['p(Kurt)'][6]=np.round(st.kurtosistest(s5accu50dt)[1],3)

stats50['Skew'][6]=np.round(st.skew(s5accu50dt),3)
if np.round(st.skewtest(s5accu50dt)[1],3) < 0.001:
    stats50['p(Skew)'][6]='p<0.001'
else:
    stats50['p(Skew)'][6]=np.round(st.skewtest(s5accu50dt)[1],3)

In [ ]:
#2016-2020
stats15=stats[:7].copy()
for i in np.arange(5):
    #cmip ensemble stats
    ind = np.arange(len(c15s5_accudt[i]))  
    val = c15s5_accudt[i]   
    ind = sm.add_constant(ind)
    reg = sm.OLS(val, ind).fit()
    stats15['Durbin-Watson'][i]=np.round(durbin_watson(reg.resid),3)
    
    stats15['Kurtosis'][i]=np.round(st.kurtosis(c15s5_accudt[i]),3)
    if np.round(st.kurtosistest(c15s5_accudt[i])[1],3) < 0.001:
        stats15['p(Kurt)'][i]='p<0.001'
    else:
        stats15['p(Kurt)'][i]=np.round(st.kurtosistest(c15s5_accudt[i])[1],3)
    stats15['Skew'][i]=np.round(st.skew(c15s5_accudt[i]),3)
    if np.round(st.skewtest(c15s5_accudt[i])[1],3) < 0.001:
        stats15['p(Skew)'][i]='p<0.001'
    else:
        stats15['p(Skew)'][i]=np.round(st.skewtest(c15s5_accudt[i])[1],3)

#ensemble stats
ind = ind = np.tile(np.arange(365),5)
val = cm15ens   
ind = sm.add_constant(ind)
reg = sm.OLS(val, ind).fit()
stats15['Durbin-Watson'][5]=np.round(durbin_watson(reg.resid),3)

stats15['Kurtosis'][5]=np.round(st.kurtosis(cm15ens),3)
if np.round(st.kurtosistest(cm15ens)[1],3) < 0.001:
    stats15['p(Kurt)'][5]='p<0.001'
else:
    stats15['p(Kurt)'][5]=np.round(st.kurtosistest(cm15ens)[1],3)

stats15['Skew'][5]=np.round(st.skew(cm15ens),3)
if np.round(st.skewtest(cm15ens)[1],3) < 0.001:
    stats15['p(Skew)'][5]='p<0.001'
else:
    stats15['p(Skew)'][5]=np.round(st.skewtest(cm15ens)[1],3)
    
#SO3
ind = np.arange(len(s5accu15dt))  
val = s5accu15dt
ind = sm.add_constant(ind)
reg = sm.OLS(val, ind).fit()
stats15['Durbin-Watson'][6]=np.round(durbin_watson(reg.resid),3)

stats15['Kurtosis'][6]=np.round(st.kurtosis(s5accu15dt),3)
if np.round(st.kurtosistest(s5accu15dt)[1],3) < 0.001:
    stats15['p(Kurt)'][6]='p<0.001'
else:
    stats15['p(Kurt)'][6]=np.round(st.kurtosistest(s5accu15dt)[1],3)

stats15['Skew'][6]=np.round(st.skew(s5accu15dt),3)
if np.round(st.skewtest(s5accu15dt)[1],3) < 0.001:
    stats15['p(Skew)'][6]='p<0.001'
else:
    stats15['p(Skew)'][6]=np.round(st.skewtest(s5accu15dt)[1],3)

In [ ]:
#2091-2095
stats90=stats[:7].copy()
for i in np.arange(5):
    #cmip ensemble stats
    ind = np.arange(len(c90s5_accudt[i]))  
    val = c90s5_accudt[i]   
    ind = sm.add_constant(ind)
    reg = sm.OLS(val, ind).fit()
    stats90['Durbin-Watson'][i]=np.round(durbin_watson(reg.resid),3)
    
    stats90['Kurtosis'][i]=np.round(st.kurtosis(c90s5_accudt[i]),3)
    if np.round(st.kurtosistest(c90s5_accudt[i])[1],3) < 0.001:
        stats90['p(Kurt)'][i]='p<0.001'
    else:
        stats90['p(Kurt)'][i]=np.round(st.kurtosistest(c90s5_accudt[i])[1],3)
    stats90['Skew'][i]=np.round(st.skew(c90s5_accudt[i]),3)
    if np.round(st.skewtest(c90s5_accudt[i])[1],3) < 0.001:
        stats90['p(Skew)'][i]='p<0.001'
    else:
        stats90['p(Skew)'][i]=np.round(st.skewtest(c90s5_accudt[i])[1],3)

#ensemble stats
ind = ind = np.tile(np.arange(365),5)
val = cm90ens   
ind = sm.add_constant(ind)
reg = sm.OLS(val, ind).fit()
stats90['Durbin-Watson'][5]=np.round(durbin_watson(reg.resid),3)

stats90['Kurtosis'][5]=np.round(st.kurtosis(cm90ens),3)
if np.round(st.kurtosistest(cm90ens)[1],3) < 0.001:
    stats90['p(Kurt)'][5]='p<0.001'
else:
    stats90['p(Kurt)'][5]=np.round(st.kurtosistest(cm90ens)[1],3)

stats90['Skew'][5]=np.round(st.skew(cm90ens),3)
if np.round(st.skewtest(cm90ens)[1],3) < 0.001:
    stats90['p(Skew)'][5]='p<0.001'
else:
    stats90['p(Skew)'][5]=np.round(st.skewtest(cm90ens)[1],3)
    
#SO3
ind = np.arange(len(s5accu90dt))  
val = s5accu90dt
ind = sm.add_constant(ind)
reg = sm.OLS(val, ind).fit()
stats90['Durbin-Watson'][6]=np.round(durbin_watson(reg.resid),3)

stats90['Kurtosis'][6]=np.round(st.kurtosis(s5accu90dt),3)
if np.round(st.kurtosistest(s5accu90dt)[1],3) < 0.001:
    stats90['p(Kurt)'][6]='p<0.001'
else:
    stats90['p(Kurt)'][6]=np.round(st.kurtosistest(s5accu90dt)[1],3)

stats90['Skew'][6]=np.round(st.skew(s5accu90dt),3)
if np.round(st.skewtest(s5accu90dt)[1],3) < 0.001:
    stats90['p(Skew)'][6]='p<0.001'
else:
    stats90['p(Skew)'][6]=np.round(st.skewtest(s5accu90dt)[1],3)

In [ ]:
stats50.to_csv(savepath+'stats_1951-1955.csv')
stats15.to_csv(savepath+'stats_2016-2020.csv')
stats90.to_csv(savepath+'stats_2091-2095.csv')